# MLB 85/1-4-3

In [1]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
# pd.options.display.mpl_style = 'default'

con = sqlite3.connect("../data/CongoDB.sqlite")

## Funde

In [2]:
sql = """SELECT
           t_Obj.ObjID, 
           t_Obj.ort_kurz, 
           t_Obj.Komplex, 
           t_Obj.Individuum,
           t_Obj.Scherbe,
           t_Obj.Typ, 
           t_Obj.Gewicht,
           t_Obj.Anzahl,
           t_Obj.Gr_Clist, 
           t_Obj.Art, 
           t_Obj.Form_Mdg, 
           t_Obj.Form_Rand
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) = 'MLB')
           AND ((t_Komplex.bef_nr) = '85/1-4-3'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,objID,ort_kurz,Komplex,Individuum,Scherbe,Typ,Gewicht,Anzahl,Gr_Clist,Art,Form_Mdg,Form_Rand
0,1978,MLB,85/1-4-3,1,W,BTM,39,1,70,K,,None
1,1979,MLB,85/1-4-3,2,R,None,4,1,30,K,M1,A1/sehr kurz
2,1980,MLB,85/1-4-3,3,W,None,10,1,70,K,,B2/C2 (?)
3,1981,MLB,85/1-4-3,4,W,BTM (?),6,1,30,K,,B2
4,1982,MLB,85/1-4-3,5,W,BTM (?),7,1,30,K,,None
5,1983,MLB,85/1-4-3,6,W,BTM (?),5,1,30,K,,None
6,1984,MLB,85/1-4-3,,W,None,5,1,30,K,,None


Summe (Gewicht)

In [3]:
df['Gewicht'].sum()

76

Scherben-Typen:

In [4]:
df_pivot = df.pivot_table(values = 'objID', index = 'Scherbe', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)
df_pivot

,objID
Scherbe,
R,1
W,6


In [5]:
df_pivot1 = df.pivot_table(values = 'Gewicht', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot1 = pd.DataFrame(df_pivot1)
# df_pivot1 = df_pivot1.reset_index()
df_pivot1.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot1['\%'] = (df_pivot1['Gewicht']/df_pivot1['Gewicht'].sum()*100).round(1)
df_pivot1['Gewicht'] = (df_pivot1['Gewicht'] / 1000).round(2)
df_pivot1.rename(columns={'Gewicht': 'Gewicht (kg)'}, inplace=True)

df_pivot2 = df.pivot_table(values = 'Anzahl', index = 'Art', aggfunc = sum, fill_value = 0)
df_pivot2 = pd.DataFrame(df_pivot2)
df_pivot2.rename(index={'HL': 'gebrannter Lehm', 'K': 'Keramik'}, inplace=True)
df_pivot2['\%'] = (df_pivot2['Anzahl']/df_pivot2['Anzahl'].sum()*100).round(1)

# Tabellen vereinen:
df_concat = pd.concat([df_pivot2, df_pivot1], axis = 1)
df_concat = df_concat.reset_index()
df_concat.rename(columns={'Art': 'Fundkategorie'}, inplace=True)
df_concat.to_latex('../output/tabs/9-03_MLB85-1-4-3_Funde.tex', index = False, escape = False)
df_concat

,Fundkategorie,Anzahl,\%,Gewicht (kg),\%
0,Keramik,7,100.0,0.08,100.0


## Formen

In [6]:
sql = """SELECT
           t_Obj.ObjID, 
           t_Obj.Typ, 
           t_Obj.Scherbe,
           t_Obj.Form_Mdg, 
           t_Obj.Form_Rand,
           t_Obj.Form_Hals,
           t_Obj.Form_Schulter,
           t_Obj.Form_Bauch,
           t_Obj.Form_Boden,
           t_Obj.Form_Gef
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_kurz) = 'MLB')
           AND ((t_Komplex.bef_nr) = '85/1-4-3'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,objID,Typ,Scherbe,Form_Mdg,Form_Rand,Form_Hals,Form_Schulter,Form_Bauch,Form_Boden,Form_Gef
0,1978,BTM,W,,None,B2,A2,,,G7b
1,1979,None,R,M1,A1/sehr kurz,,,,,
2,1980,None,W,,B2/C2 (?),,,,,
3,1981,BTM (?),W,,B2,A1/A2,,,,
4,1982,BTM (?),W,,None,A2 (?),A2 (?),,,
5,1983,BTM (?),W,,None,,,,,
6,1984,None,W,,None,,,,,


## 14C-Datierungen

In [78]:
sql = """SELECT
           t_14C.LABNR,
           t_14C.C14AGE,
           t_14C.C14STD,
           t_14C.C13,
           t_14C.MATERIAL,
           t_14C.SPECIES,
           [t_Ort].[ort_name] AS SITE, 
           [t_Komplex].[bef_art] AS CULTURE, 
           t_Komplex.bef_KeramikBeschr AS POTTERY,
           [t_Ort].[ort_kurz] || ' ' || [t_Komplex].[bef_nr] AS PHASE,
           [t_ort].[ort_land] AS COUNTRY,
           [t_ort].[y_lat] AS LATITUDE,
           [t_ort].[x_long] AS LONGITUDE
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_14c ON t_Komplex.komplexID = t_14c.komplexID
       WHERE (((t_Ort.ort_kurz) = 'MLB')
           AND ((t_Komplex.bef_nr) = '85/1-4-3'))""".replace('\n',' ')

df = pd.read_sql(sql, con)
df

,LABNR,C14AGE,C14STD,C13,MATERIAL,SPECIES,SITE,CULTURE,POTTERY,PHASE,COUNTRY,LATITUDE,LONGITUDE
0,Poz-62102,580,30,0,Bone,None,Maluba,Grab,,MLB 85/1-4-3,COD,2.806193,18.523706
1,Poz-62103,810,80,0,Bone,None,Maluba,Grab,,MLB 85/1-4-3,COD,2.806193,18.523706


mlab.normpdf verwendet die Standartabweichung der Verteilung. Die Standartabweichung errechnet aus dem gegeben Standartfehler folgendermaßen:

$$SD = SE \sqrt{N}$$

> siehe http://handbook.cochrane.org/chapter_7/7_7_3_2_obtaining_standard_deviations_from_standard_errors_and.htm

Bei einem n = 1 entspricht die Standartabweichung dem Standartfehler. Dieser kann somit weiter verwendet werden. 

In [107]:
import matplotlib.mlab as mlab
import math

mu1 = df['C14AGE'][0]
mu2 = df['C14AGE'][1]
sigma1 = df['C14STD'][0]
sigma2 = df['C14STD'][1]
x = np.linspace(400, 1100, 100)


plt.plot(x1, mlab.normpdf(x, mu1, sigma1), color = 'g')
plt.plot(x, mlab.normpdf(x, mu2, sigma2), color = 'r')

plt.show()